# motive
A. explore mutiple way to build agent so that we can show how flexible we are --  
1. fully automate
2. semi automate
3. manual user entry


B. make multitasking tool
1. search
2. source
3. rdf generation
4. synthesizer
## notebook work plan

1. defining all tool (google, duck duck, wiki, source, synthesize)
2. reading excel where query stored
3. iterate throught excel and call agent for information
4. save response in new column named "result"

In [ ]:
from langchain.memory import ConversationBufferMemory
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
# from langchain.tools import DuckDuckGoSearchTool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.vectorstores import Chroma
import re
import openai
import ast
import pandas as pd

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-QCTEMIILQ"
#
llm4=OpenAI(temperature=0, model_name="gpt-4-1106-preview")
            #4-1106-preview")
embed_model = OpenAIEmbeddings()

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# collecting initial company information

In [ ]:
!pip install -q openai langchain beautifulsoup4 html2text
# !playwright install   playwright
!pip -q install langchain tiktoken pypdf chromadb
!pip install nltk
!python -m nltk.downloader punkt

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/typing_extensions-4.7.1.dist-info/METADATA'

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/typing_extensions-4.7.1.dist-info/METADATA'

/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pandas as pd
company_df = pd.DataFrame(columns=[ 'url','full_text','sentence', 'Capability', 'Keywords'])
def update_dataframe(company_df, url, full_text, sentence, Capability, Keywords):
    new_data = {
        'url': url,
        'full_Text': full_text,
        'sentence': sentence,
        'Keywords': Keywords,
        'Capability': Capability
    }

    # Append new data to DataFrame
    company_df = company_df.append(new_data, ignore_index=True)

    return company_df

In [ ]:
prompt1=f"""
you are an expert in building Resource Description Framework (RDF).
so from the given sentence, give me a relationship in the following format.

Here are two examples:
example 1: {{'sentence': 'Lithium-ion batteries have dominated the market for years.', 'Capability': 'Battery Type', 'Keywords': ['Lithium-ion', 'Solid-state', 'Flow battery']}}
example 2: {{'sentence': 'Solid-state batteries promise a higher energy density.', 'Capability': 'Energy Density', 'Keywords': ['High', 'Low', None]}}

Now, give me an answer for the sentence:

"""


def rdf_find(text):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt1},
            {"role": "user", "content": text}
        ]
    )

    output = response.choices[0].message.content
    print("rdf output is ", output)


    data_dict = ast.literal_eval(output)
    # Check if 'Keywords' is in the dictionary and is not None
    if 'Keywords' in data_dict and data_dict['Keywords'] is not None:
        # Ensure all items in the list are strings
        data_dict['Keywords'] = ', '.join(str(kw) for kw in data_dict['Keywords'] if kw is not None)
    else:
        data_dict['Keywords'] = ''

    # Concatenate the new data to the DataFrame
    new_row = pd.DataFrame([data_dict])
    global company_df
    company_df = pd.concat([company_df, new_row], ignore_index=True)

    return output


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def preprocess_text_list(text_list):
    # Remove all punctuation except commas and periods
    translator = str.maketrans('', '', string.punctuation.replace(',', '').replace('.', ''))
    preprocessed_texts = text_list.translate(translator)

    return preprocessed_texts




In [ ]:
import nest_asyncio
nest_asyncio.apply()

# from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import BeautifulSoupTransformer
from langchain.document_transformers import Html2TextTransformer

# Load HTML
def scrap(url):
    loader = AsyncHtmlLoader([url])
    html = loader.load()

    # Transform
    # bs_transformer = BeautifulSoupTransformer()
    # docs_transformed = bs_transformer.transform_documents(html, tags_to_extract=["span"])
    docs=html
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)

    #splitting the text into
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    texts = text_splitter.split_documents(docs_transformed)
    print(len(texts))


    # adding synthetic data paper
    embedding=OpenAIEmbeddings()
    vdb = FAISS.from_documents(texts, embedding)
    retriever = vdb.as_retriever(search_kwargs={"k": 4})
    # create the chain to answer questions
    pdf_tool = RetrievalQA.from_chain_type(llm=llm4,
                                      chain_type="stuff",
                                      retriever=retriever,
                                      return_source_documents=True)
    res= pdf_tool("""give me key capablity.
                future plans.
                financial informations.
                strategies.

                """)
    full_text=res['result']
    global company_df
    # Update the DataFrame with the sample data
    data_dict = {'url': url, 'full_text': full_text}
    new_row = pd.DataFrame([data_dict])

    company_df = pd.concat([company_df, new_row], ignore_index=True)
    from nltk.tokenize import sent_tokenize

    print("### full text ",full_text)

    preprocessed_texts = preprocess_text_list(full_text)
    # Process each response
    print("### preprocessed_texts ",preprocessed_texts)

    for response in preprocessed_texts.split('.'):
        # Split the response into sentences
        # Check if the response is not an empty string
        if response.strip():
            sentences = sent_tokenize(response)
            print("### response  ",response)

            # Process each sentence
            for sentence in sentences:
                print("### sentence ", sentence)
                rdf_find(sentence)

    return res['result']


In [ ]:
# company_df['full_text']
company_df

In [ ]:
company_in_data("microsoft")

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='./chroma_db_oai')

web_research_retriever = WebResearchRetriever.from_llm(
                            vectorstore=vectorstore,
                            llm=llm4,
                            search=search_google  )

def company_in_data(company_name):

    pr= f"""give me {company_name} company information like
                key capablity.
                future plans.
                financial informations.
                strategies.
                website links"""
    source = RetrievalQAWithSourcesChain.from_chain_type(llm=llm4, retriever=web_research_retriever)
    info= source(pr)
    text =  info['answer']
    # Use a regular expression to find all URLs in the text
    urls = re.findall(r'https?://\S+', text)

    # Print the extracted URLs
    for url in urls:
        print(url)
        scrap(url)

    return urls

# source_tool = Tool(name="source_tool", description = "Useful for when you need to search from the internet with website links or sources ",func = source)

In [ ]:
company_in_data("apple")

In [ ]:
source = RetrievalQAWithSourcesChain.from_chain_type(llm=llm4, retriever=web_research_retriever)

source("""give me apple company information like
                key capablity.
                future plans.
                financial informations.
                strategies.
                website links""")

In [ ]:
print("hi")

hi


# auto agent

## google search

In [ ]:
import os

os.environ["GOOGLE_CSE_ID"] = "816ee"
os.environ["GOOGLE_API_KEY"] = "AIzaSyAgEIc"
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search_google = GoogleSearchAPIWrapper()

gs_tool = Tool(
    name="Google Search",
    description="Search Google for recent results. give where from you collecting data all references",
    func=search_google.run,
)

## duck duck

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search_duck = DuckDuckGoSearchRun()
search_tool_duck = Tool(name="search_tool_duck", description = "Useful for when you need to search the net for current events,latest informations.give where from you collecting data all references ",func = search_duck.run)


## wiki

In [ ]:
!pip install -q wikipedia
from langchain.utilities import WikipediaAPIWrapper
wikipedia = WikipediaAPIWrapper()
wiki = Tool(name="wiki", description = "Useful for when you need to search for all basic information , references, history.give where from you collecting data all references",func = wikipedia.run)

# source tool

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='./chroma_db_oai')

web_research_retriever = WebResearchRetriever.from_llm(
                            vectorstore=vectorstore,
                            llm=llm4,
                            search=search_google  )

source= RetrievalQAWithSourcesChain.from_chain_type(llm=llm4,retriever=web_research_retriever)

source_tool = Tool(name="source_tool", description = "Useful for when you need to search from the internet with website links or sources ",func = source)

In [ ]:
source_tool("battery technology")

how many website link do you want? 4



Fetching pages: 100%|##########| 3/3 [00:00<00:00,  4.53it/s]


{'question': 'battery technology I want 4 website links for this topic',
 'answer': 'Here are four website links related to battery technology:\n\n1. "Three battery technologies that could power the future" - Saft\n   - Link: [https://www.saft.com/media-resources/our-stories/three-battery-technologies-could-power-future](https://www.saft.com/media-resources/our-stories/three-battery-technologies-could-power-future)\n\n2. "What\'s next for batteries?" - MIT Technology Review\n   - Link: [https://www.technologyreview.com/2023/01/04/1066141/whats-next-for-batteries/](https://www.technologyreview.com/2023/01/04/1066141/whats-next-for-batteries/)\n\n3. "New generation lithium-ion batteries" - Saft\n   - Link: [https://www.saft.com/media-resources/our-stories/three-battery-technologies-could-power-future](https://www.saft.com/media-resources/our-stories/three-battery-technologies-could-power-future)\n\n4. "Lithium-sulfur batteries" - Saft\n   - Link: [https://www.saft.com/media-resources/our

In [ ]:
res=source_tool("solar energy ")
print(res['answer'])

how many website link do you want? 2



Fetching pages: 100%|##########| 3/3 [00:01<00:00,  2.57it/s]


Here are two website links on the topic of solar energy:

1. "Solar Energy Pros and Cons" - https://www.constellation.com/energy-101/energy-innovation/solar-energy-pros-and-cons.html
2. "Advancements in Solar Energy Technology" - https://www.wellspring.com/blog/advancements-in-solar-energy-technology




# synthesize tool

In [ ]:
import openai
def synthesize(query):
    global final_responses

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "you are a helpfull assistant"},
            {"role": "user", "content": query}
        ]
    )
    final_responses=[]
    final_responses.append(response.choices[0].message.content)
    return final_responses
synthesize_tool = Tool(name="synthesize_tool", description = "Useful for when you need to synthesize information ",func = synthesize)


In [ ]:
synthesize_tool("lead acid is best")

do you want full response synthesize if yes enter y otherwise n n
how you want to change thge response ? add lithium is good


'Lithium is indeed considered beneficial for certain applications. It is commonly used in batteries, especially in rechargeable lithium-ion batteries, due to its high energy density and long lifespan. Lithium is also used in the treatment of mental health conditions such as bipolar disorder, where it can help stabilize mood and reduce symptoms. However, it is important to note that lithium should only be used under the supervision and guidance of a healthcare professional, as it can have potential side effects and interactions with other medications.'

In [ ]:
synthesize_tool("lead acid is best")

how you want to change thge response ? add moon and mars


'Battery technology advancements can also impact space exploration missions to the Moon and Mars. In order to supply power to spacecraft and habitats in these extraterrestrial environments, battery systems need to be highly reliable, long-lasting, and capable of functioning in extreme conditions. Research is being conducted to develop batteries specifically designed for space applications, including those that can withstand radiation, low temperatures, and long-duration missions. These advancements will help enable sustainable human presence and scientific exploration on the Moon and Mars by providing reliable power sources for critical equipment and systems.'

# define agent

In [ ]:
from langchain.prompts import PromptTemplate


tools = [wiki, search_tool_duck, gs_tool, source_tool, synthesize_tool]

 # PromptTemplate(input_variables=["tools"], template=template, )


# create our agent search_tool_duck
conversational_agent = initialize_agent(tools=tools,
                                        llm=llm4,
                                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                        # ZERO_SHOT_REACT_DESCRIPTION,
                                        # CONVERSATIONAL_REACT_DESCRIPTION,
                                        max_iteration=3,
                                        early_stopping_method='generate',
                                        # memory=memory,
                                        verbose=True,
                                        handle_parsing_errors=True,
                                        # prompt=search_prompt,
                                        )



In [ ]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 6.6 MB/s eta 0:00:0000:01


In [ ]:
df_comp = pd.read_excel('company.xlsx')

In [ ]:
df_comp

,query,engine,page,result
0,apple social initiative,google,5,NaN
1,apple market analysis,google,3,NaN
2,facebook social initiative,google,4,NaN
3,microsoft market analysis,wiki,3,NaN
4,microsoft social initiative,google,5,NaN
5,meta market analysis,wiki,3,NaN
6,meta social initiative,duck,3,NaN
7,samsung market analysis,google,3,NaN
8,samsung social initiative,google,5,NaN


## 1st way no control over tool
* very slow

In [ ]:
# Iterate over the DataFrame rows
for index, row in df_comp.iterrows():

    query=f"""use {row['engine']} tool I want to know about {row['query']} give me {row['page']} page answer"""
    print(query)
    # Call the agent function with the data from the row
    response = conversational_agent(query)

    # Store the response in the 'result' column
    df_comp.at[index, 'result'] = response['output']
    print(df_comp)

use google tool I want to know about apple social initiative  give me 5 page answer


> Entering new AgentExecutor chain...
To provide a comprehensive answer about Apple's social initiatives, I will need to gather information from various sources, including Apple's official website and other credible sources that detail their social responsibility programs and efforts.

Action: Google Search
Action Input: Apple social initiatives
Observation: Advancing equity today. And every day. · Education that powers possibility. We're increasing support for minority-serving institutions and programs that foster ... At Apple, we work every day to put people first — by empowering them with accessible technology, being a force for equity and opportunity, creating an inclusive ... Lisa Jackson is Apple's vice president of Environment, Policy and Social Initiatives, and a member of Apple's executive team. Learn more about Lisa. Jul 11, 2023 ... Apple Corporate Social Responsibility (CSR) · Apple Suppor

Fetching pages: 100%|##########| 2/2 [00:01<00:00,  1.55it/s]



Observation: {'question': "I searched for the Apple ESG Report 2022 on Apple's official website.", 'answer': "The Apple ESG Report 2022 can be found on Apple's official website. The report and index detail Apple's commitment to environmental, social, and governance (ESG) initiatives, including their goal to make carbon-neutral products by 2030 and their efforts in green chemistry and recycling. The report also outlines Apple's dedication to creating an inclusive and diverse work environment, respecting human rights, and ensuring transparency and accountability at every level of the company. Additionally, the report is mapped across metrics outlined in the Global Reporting Initiative (GRI), Sustainable Accounting Standards Board (SASB), and the Task Force on Climate-Related Financial Disclosures (TCFD) voluntary disclosure frameworks.\n\n", 'sources': 'https://investor.apple.com/esg/default.aspx, https://www.apple.com/environment/'}
Thought:I have found the source for the Apple ESG Rep

KeyboardInterrupt: 

In [ ]:
print(df_comp['result'][0])

Apple's ESG Report 2022 outlines several key social initiatives that the company is actively pursuing:

1. **Employee Well-being**: Apple is committed to fostering a safe, inclusive, and respectful work environment for its diverse workforce. The company focuses on diversity and inclusion, providing equal opportunities and promoting a culture that values every employee.

2. **Supply Chain Responsibility**: The company ensures fair and safe working conditions for workers throughout its supply chain. Apple works with suppliers to maintain responsible sourcing practices, emphasizing worker empowerment and human rights protection.

3. **Education and Skill-Building**: Through programs like Everyone Can Code and Everyone Can Create, Apple provides educators with resources to integrate coding and creativity into their curriculum, aiming to inspire and empower students with new learning opportunities.

4. **Community Engagement**: Apple partners with local organizations to leverage technology 

# 2nd way controlling tool

In [ ]:
tool_options = {
    "google": gs_tool,
    "duck": search_tool_duck,
    "wiki": wiki,
    "sources": source_tool,
    "synthesize": synthesize_tool,
}


# Iterate over the DataFrame rows
for index, row in df_comp.iterrows():

    query=f"""use {row['engine']} tool.
    I want to know about {row['query']}.
    give me {row['page']} page answer for this topic."""

    print(query)
    selected_tool = tool_options.get(row['engine'])

    tools = [selected_tool]

    # create our agent search_tool_duck
    conversational_agent = initialize_agent(tools=tools, llm=llm4, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  max_iteration=3,
                                            early_stopping_method='generate', verbose=True,  handle_parsing_errors=True, )

    # Call the agent function with the data from the row
    response = conversational_agent(query)
    print(response['output'])
    # Store the response in the 'result' column
    df_comp.at[index, 'result'] = response['output']


use google tool.
    I want to know about apple social initiative .
    give me 5 page answer for this topic.


> Entering new AgentExecutor chain...
To gather information about Apple's social initiatives, I will need to perform a Google Search to find relevant and recent sources that discuss the topic in detail. Since the user has requested a 5-page answer, I will aim to provide a comprehensive summary of the information found, which may include various aspects of Apple's social initiatives such as environmental efforts, education programs, diversity and inclusion policies, and community outreach.

Action: Google Search
Action Input: Apple social initiatives
Observation: Advancing equity today. And every day. · Education that powers possibility. We're increasing support for minority-serving institutions and programs that foster ... At Apple, we work every day to put people first — by empowering them with accessible technology, being a force for equity and opportunity, creating an incl

In [ ]:
print(df_comp)

                          query  engine  page  \
0      apple social initiative   google     5   
1        apple market analysis   google     3   
2   facebook social initiative   google     4   
3    microsoft market analysis     wiki     3   
4  microsoft social initiative   google     5   
5         meta market analysis     wiki     3   
6       meta social initiative     duck     3   
7      samsung market analysis   google     3   
8    samsung social initiative   google     5   

                                              result  
0  Apple's social initiatives encompass a broad r...  
1  Due to the limitations of this platform, I can...  
2  To create a 4-page answer on Facebook's social...  
3  Microsoft is a leading technology company with...  
4  To write a 5-page answer on Microsoft's social...  
5  I am unable to provide information on "meta ma...  
6  Meta's social initiatives encompass a range of...  
7  Based on the information gathered from the sea...  
8  Samsung's s

In [ ]:
print(df_comp['result'][0])

Apple's social initiatives encompass a broad range of efforts aimed at creating positive social change and promoting sustainability. Key areas of focus include:

1. Environmental Sustainability: Apple has set ambitious goals to reduce its carbon footprint, aiming to make all its products carbon neutral by 2030. The company is working on cutting down plastic packaging, creating renewable energy projects, and supporting local organizations that promote environmental justice.

2. Education and Community Programs: Apple supports education through initiatives like the Apple Education Community, a professional learning hub for educators, and the Community Education Initiative, which provides hardware and scholarships to schools and educational institutions. The Apple Teacher program offers professional learning for educators, and Apple supports curriculum development and various educational programs globally.

3. Diversity and Inclusion: Apple is committed to fostering a diverse and inclusiv

In [ ]:
print(df_comp['result'][6])

Meta's social initiatives encompass a range of efforts aimed at responsible business practices, including data privacy and security, human rights, content governance, trust and integrity, diversity, equity and inclusion, and the responsible design of apps and technologies. They have released a Responsible Business Practices Report that outlines four pillars: operating transparently and sustainably, empowering people and partners, building safe and inclusive communities, and driving social impact. Additionally, Meta is focused on attracting and retaining younger users while ensuring their safety online, investing in AI and metaverse technologies, and advocating for clear, consistent global regulation for online experiences. To create a detailed 3-page document, one would need to elaborate on each of these points, providing specific examples of initiatives, their implementation, and the outcomes or goals Meta aims to achieve.


In [ ]:
print(df_comp['result'][8])

Samsung's social initiatives include:

1. Global Human Rights Principles: Announced in February 2023, these principles demonstrate Samsung's commitment to respecting human rights.
2. Training Initiatives: Samsung focuses on building technical capabilities through diverse training programs.
3. Samsung Innovation Campus (SIC): A global CSR initiative aimed at upskilling youth aged 18-25 in future technologies.
4. Energy-Saving Innovations: New products and solutions that help consumers save energy and reduce environmental impact.
5. Solve for Tomorrow Competition: A contest that encourages innovation and problem-solving among youth.
6. Sustainability Commitments: Samsung has made commitments to sustainable business practices and implementing initiatives to support this goal.

These initiatives reflect Samsung's dedication to empowering future generations, respecting human rights, promoting sustainability, and fostering innovation.


# old agent calling

In [ ]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain


vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory='./chroma_db_oai')
tool_options = {
    "google": gs_tool,
    "duck": search_tool_duck,
    "wiki": wiki,
    "sources": source_tool,
    "synthesize": synthesize_tool,
}


def agent_call(query, tool):
    conversational_agent.tools = [tool]  # Set the selected tool for the agent to use
    response = conversational_agent(query)  # Call the agent to iterate with the given tool and query
    return response['output']

def ask_for_tool():
    tool_name = input("""Which tool do you want to use? (google, duck, wiki, sources, synthesize): enter "exit" if you want to finish""")
    return tool_name.lower()

response=""
final_responses=[]
def new_info_agent(query):
    while True:
        selected_tool_name = ask_for_tool()

        if selected_tool_name.lower() == 'exit':
            print("Exiting the tool selection.")
            print("Response:", final_responses)
            break
        elif selected_tool_name == 'sources':
            pr=input("how many website link do you want?")
            pr=query+f" I want {pr} website links on this topic"

        elif selected_tool_name == 'synthesize':
            chan=input("how you want to change thge response ?")
            pr=''.join(final_responses) + chan
        selected_tool = tool_options.get(selected_tool_name)
        # print("tool name ",selected_tool)
        if selected_tool:
            if selected_tool_name == 'sources':
                response=agent_call(pr,selected_tool)
            elif selected_tool_name == 'synthesize':
                response=agent_call(pr,selected_tool)
            else:
                response=agent_call(query,selected_tool)
            final_responses.append(response)
            print("Response:", response)
            qry=input("input new query or type same for existing query")
            if qry != 'same':
                query=qry

        else:
            print("Unknown tool. Please choose between 'google', 'duckduckgo', or 'wikipedia'.")


In [ ]:
new_info_agent("battery technology")

Which tool do you want to use? (google, duck, wiki, sources, synthesize): enter "exit" if you want to finish duck




> Entering new AgentExecutor chain...
I should start by searching for basic information on battery technology.
Action: wiki
Action Input: battery technology
Observation: wiki is not a valid tool, try one of [search_tool_duck].
Thought:I should try using the search_tool_duck to find information on battery technology.
Action: search_tool_duck
Action Input: battery technology
Observation: A radical rethink Some dramatically different approaches to EV batteries could see progress in 2023, though they will likely take longer to make a commercial impact. One advance to keep an eye on... November 8, 2023 Top Headlines Making Electric Vehicles Last Nov. 2, 2023 — In the realm of electric vehicles, powered by stored electric energy, the key lies in rechargeable batteries capable of... Introduction to battery technology Simply put, the modern world as we know it would not be possible without batteries. From life-sustaining devices like pacemakers to the cellphone, batteries... Energy Saver How

input new query or type same for existing query same
Which tool do you want to use? (google, duck, wiki, sources, synthesize): enter "exit" if you want to finish synthesize
how you want to change thge response ? add moon




> Entering new AgentExecutor chain...
I need to gather information about battery technology, including the current most common type of battery used and ongoing research and development efforts.
Action: wiki
Action Input: Battery technology
Observation: wiki is not a valid tool, try one of [synthesize_tool].
Thought:I should try using the synthesize_tool to gather information about battery technology.
Action: synthesize_tool
Action Input: Battery technology
Observation: ['Battery technology refers to the science and engineering behind the development, design, and optimization of batteries. Batteries are electrochemical devices that store and release electrical energy through a chemical reaction.\n\nThere are various types of batteries, each with its own advantages and limitations. Here are a few common types:\n\n1. Lithium-ion (Li-ion) Batteries: These are widely used in portable electronic devices and electric vehicles. Li-ion batteries have high energy density and long cycle life.\n

input new query or type same for existing query same
Which tool do you want to use? (google, duck, wiki, sources, synthesize): enter "exit" if you want to finish sources
how many website link do you want? 3


...


Observation: {'question': 'solar power as sustainable energy website links', 'answer': 'Here are some website links related to solar power as sustainable energy:\n\n1. "Advantages and Challenges of Solar Energy" - This article discusses the advantages of solar energy technology, including its renewable and clean nature, cost-effectiveness, low maintenance, and environmental friendliness. It also emphasizes the importance of solar energy in ensuring sustainable development. [Source](https://academic.oup.com/ce/article/6/3/476/6606003)\n\n2. "Installed Capacity and Application of Solar Energy Worldwide" - This article provides an overview of the installed capacity of solar energy globally and its various applications. It highlights the role of solar energy in sustainable development and the employment of renewable energy. [Source](https://academic.oup.com/ce/article/6/3/476/6606003)\n\n3. "Solar Energy Pros and Cons" - This webpage discusses the benefits of solar energy for homeowners. 

input new query or type same for existing query same
Which tool do you want to use? (google, duck, wiki, sources, synthesize): enter "exit" if you want to finish synthesize
how you want to change thge response ? add paragraph about quantum evolution after openai gpt model




> Entering new AgentExecutor chain...
I need to gather information on battery technology, specifically the different types of batteries and their advantages and limitations.
Action: wiki
Action Input: Battery technology
Observation: wiki is not a valid tool, try one of [synthesize_tool].
Thought:I should try using the synthesize_tool to gather information on battery technology.
Action: synthesize_tool
Action Input: Battery technology
Observation: ['Battery technology refers to the development and improvement of batteries, which are devices that store and release electrical energy. Batteries are commonly used in a variety of applications, including electronic devices, electric vehicles, and renewable energy systems.\n\nOver the years, battery technology has advanced significantly, with major developments in terms of capacity, efficiency, and reliability. There are several types of batteries, each with its own characteristics and advantages:\n\n1. Lithium-ion batteries: This is current

input new query or type same for existing query same
Which tool do you want to use? (google, duck, wiki, sources, synthesize): enter "exit" if you want to finish exit


Exiting the tool selection.
Response: ['Battery technology refers to the development and improvement of batteries, which are devices that store and release electrical energy. Batteries are commonly used in a variety of applications, including electronic devices, electric vehicles, and renewable energy systems.\n\nOver the years, battery technology has advanced significantly, with major developments in terms of capacity, efficiency, and reliability. There are several types of batteries, each with its own characteristics and advantages:\n\n1. Lithium-ion batteries: This is currently the most prevalent and popular battery technology. Lithium-ion batteries offer high energy density, long cycle life, and low self-discharge rates. They are commonly used in smartphones, laptops, and electric vehicles.\n\n2. Lead-acid batteries: These batteries have been in use for a long time and are widely used in applications such as car starting, backup power supplies, and renewable energy systems. They of

In [ ]:
final_responses

['Battery technology refers to the development and improvement of batteries, which are devices that store and release electrical energy. Batteries are commonly used in a variety of applications, including electronic devices, electric vehicles, and renewable energy systems.\n\nOver the years, battery technology has advanced significantly, with major developments in terms of capacity, efficiency, and reliability. There are several types of batteries, each with its own characteristics and advantages:\n\n1. Lithium-ion batteries: This is currently the most prevalent and popular battery technology. Lithium-ion batteries offer high energy density, long cycle life, and low self-discharge rates. They are commonly used in smartphones, laptops, and electric vehicles.\n\n2. Lead-acid batteries: These batteries have been in use for a long time and are widely used in applications such as car starting, backup power supplies, and renewable energy systems. They offer high surge currents, low cost, and

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def preprocess_text_list(text_list):
    preprocessed_texts = []

    for text in text_list:
        # Lowercasing
        text = text.lower()

        # Removing special characters, numbers, and punctuation
        text = re.sub(r"[^a-zA-Z\s']", "", text)


        preprocessed_texts.append(text)

    return preprocessed_texts

# Example usage with a list of strings
input_texts =final_responses

preprocessed_texts = preprocess_text_list(input_texts)


NameError: name 'final_responses' is not defined

# Adding capablity & keyword RDF

In [ ]:
!pip install nltk
!python -m nltk.downloader punkt


/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
cap_li=company_df["Capability"]

In [ ]:
prompt2=f"""
you are an expert in building Resource Description Framework (RDF).
so from the given sentence, give me a relationship in the following format.Ignore if it is not a proper sentence.

Here are two examples:
example 1: {{'sentence': 'Lithium-ion batteries have dominated the market for years.', 'Capability': 'Battery Type', 'Keywords': ['Lithium-ion', 'Solid-state', 'Flow battery']}}
example 2: {{'sentence': 'Solid-state batteries promise a higher energy density.', 'Capability': 'Energy Density', 'Keywords': ['High', 'Low', None]}}

You have to check one condition before generating the RDF:
Only generate RDF if the given sentence can produce a Capability from this list.
Ensure that the RDF generated Capability must be the same as in the given list.
Else, reply: {{'sentence': 'user given sentence', 'Capability': 'not matched', 'keywords': ['none']}}.

Here is the Capability list: {cap_li}.

Now, give me an answer for the sentence:

"""

In [ ]:
import ast
try:
    df_final = pd.read_csv('new_rdf.csv')
except FileNotFoundError:
    df_final = pd.DataFrame(columns=['sentence', 'Capability', 'Keywords'])

def rdf_match(text):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt2},
            {"role": "user", "content": text}
        ]
    )

    output = response.choices[0].message.content
    print("rdf output is ", output)
    print(" ")


    data_dict = ast.literal_eval(output)
    # Check if 'Keywords' is in the dictionary and is not None
    if 'Keywords' in data_dict and data_dict['Keywords'] is not None:
        # Ensure all items in the list are strings
        data_dict['Keywords'] = ', '.join(str(kw) for kw in data_dict['Keywords'] if kw is not None)
    else:
        data_dict['Keywords'] = ''

    # Concatenate the new data to the DataFrame
    new_row = pd.DataFrame([data_dict])
    global df_final
    df_final = pd.concat([df, new_row], ignore_index=True)

    return output


In [ ]:
from nltk.tokenize import sent_tokenize

# Process each response
for response in preprocessed_texts:
    # Split the response into sentences
    # print("### response from agent ",response)
    sentences = sent_tokenize(response)
    # Process each sentence

    for sentence in sentences:
        rdf_match(sentence)
        # print("### sentence ",sentence)

rdf output is  {'sentence': 'battery technology refers to the development and improvement of batteries which are devices that store and release electrical energy batteries are commonly used in a variety of applications including electronic devices electric vehicles and renewable energy systems', 'Capability': 'Advanced Manufacturing', 'Keywords': ['battery technology', 'development', 'improvement']}
 
rdf output is  {'sentence': 'battery technology refers to the development and improvement of batteries which are devices that store and release electrical energy there are several types of batteries including lithiumion leadacid nickelmetal hydride nimh and solidstate batteries each type has its own characteristics and advantages lithiumion batteries are currently the most prevalent and popular offering high energy density and long cycle life leadacid batteries have been in use for a long time and are widely used in various applications nimh batteries are rechargeable and have relatively 

In [ ]:
df_final

,sentence,Capability,Keywords,keywords
0,Battery technology refers to the development a...,Advanced Manufacturing,"Battery technology, development, improvement",NaN
1,Batteries are commonly used in a variety of ap...,not matched,,[none]
2,"Over the years, battery technology has advance...",Advanced Manufacturing,"battery technology, advanced, efficiency",NaN
3,"There are several types of batteries, each wit...",not matched,,[none]
4,Lithium-ion batteries: This is currently the m...,not matched,,[none]
5,Lithium-ion batteries offer high energy densit...,Energy Efficiency,"High energy density, Long cycle life, Low self...",NaN
6,"They are commonly used in smartphones, laptops...",not matched,,[none]
7,battery technology refers to the development a...,Materials Science,,"[batteries, development, improvement, lithium-..."


# end